In [103]:
# %additional_python_modules duckdb==1.4.2

In [104]:
# %idle_timeout 10
# %glue_version 5.0
# %worker_type G.1X
# %number_of_workers 2

# import sys
# from awsglue.transforms import *
# from awsglue.utils import getResolvedOptions
# from pyspark.context import SparkContext
# from awsglue.context import GlueContext
# from awsglue.job import Job
  
# sc = SparkContext.getOrCreate()
# glueContext = GlueContext(sc)
# spark = glueContext.spark_session
# job = Job(glueContext)

In [105]:
import duckdb
print("Installed package")

Installed package


In [106]:
import pandas as pd
import numpy as np

In [107]:
def load_raw():
    applications_train = pd.read_csv("s3://crisk-nico-prod/raw/applications/application_train.csv.gz", compression='gzip')
    print('applications_train shape: ', applications_train.shape)
    applications_test = pd.read_csv("s3://crisk-nico-prod/raw/applications/application_test.csv.gz", compression='gzip')
    print('applications_test shape: ', applications_test.shape)
    bureau = pd.read_csv("s3://crisk-nico-prod/raw/bureau/bureau.csv.gz", compression='gzip')
    print('bureau shape: ', bureau.shape)
    bureau_balance = pd.read_csv("s3://crisk-nico-prod/raw/bureau_balance/bureau_balance.csv.gz", compression='gzip')
    print('bureau_balance shape: ', bureau_balance.shape)
    return applications_train, applications_test, bureau, bureau_balance

def drop_columns(df, cols):
    # Drop the columns if they exist in the dataframe
    df.drop(columns=[col for col in cols if col in df.columns], inplace=True)
    
    print(f"Dropped {len(cols)} columns: {cols}")

def find_correlation_high_missing(df, threshold=0.8):
    """
    Finds pairs of columns with high correlation and returns the columns with the highest
    number of missing values.
    
    df: pandas DataFrame
        The dataframe to check for correlations.
        
    threshold: float
        The minimum correlation value to consider as "high".
        
    Returns:
    list: A list of column names that are highly correlated and have the most missing values.
    """
    # Calculate the correlation matrix for numeric columns and take the absolute values
    corr_matrix = df.select_dtypes(include=[np.number]).corr().abs()

    # Get the upper triangle of the correlation matrix (excluding the diagonal)
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Set to store columns to drop (those with the most missing values)
    columns_to_drop = set()

    # Iterate over each pair of correlated columns
    for col1 in upper.columns:
        for col2 in upper.index:
            if upper.loc[col2, col1] > threshold:  # If the correlation is above threshold
                # Check the missing values in both columns
                missing_col1 = df[col1].isnull().sum()
                missing_col2 = df[col2].isnull().sum()
                
                # Drop the column with more missing values
                if missing_col1 > missing_col2:
                    columns_to_drop.add(col1)
                elif missing_col1 == 0 and missing_col2 == 0:
                    break
                else:
                    columns_to_drop.add(col2)

    return list(columns_to_drop)

def high_missing(df, threshold):
    missing_ratio = df.isna().mean()
    high_missing = missing_ratio[missing_ratio > threshold].index
    return list(high_missing)

def correlation_target(df, dtype):
    nan_df = df[df.columns[df.isna().any()]]
    nan_df_columns = (nan_df.select_dtypes(include=dtype).copy()).columns
    for col in nan_df_columns:
        print(df.groupby(df[col].isna())['TARGET'].mean())

def create_missing_flag(df, cols):
    # Create missing flags for informative columns
    for col in cols:
        if col in df.columns:
            df[f'{col}_IS_MISSING'] = df[col].isna().astype('Int8')

def fill_missing_numerical(df, cols, strategy):

    if strategy == 'median': 
        # Median imputation

        for col in cols:
            df.fillna({col: df[col].median()}, inplace=True)

    elif strategy == 'zero':
        # 0-imputation for count-like features

        for col in cols:
            df.fillna({col: 0}, inplace=True)
    else:
        raise ValueError("Strategy must be 'median' or 'zero'")
        

def fill_missing_categorical(df, unknown_cols):

    # Fill missing categories with 'unknown'
    for col in unknown_cols:
        if col in df.columns:
            df[col] = df[col].fillna('Unknown')

In [108]:
applications_train, applications_test, bureau, bureau_balance = load_raw()

applications_train shape:  (307511, 122)
applications_test shape:  (48744, 121)
bureau shape:  (1716428, 17)
bureau_balance shape:  (27299925, 3)


In [109]:
# These dropped columns are totally subjective, by my own decision of dropping because
# I dont find any good information or insight about it

drop_cols = [
    'REG_REGION_NOT_LIVE_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'LIVE_CITY_NOT_WORK_CITY',
    'NAME_TYPE_SUITE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG',
    'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG',
    'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE',
    'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE',
    'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE',
    'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI',
    'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI',
    'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI',
    'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'TOTALAREA_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE',
    'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7',
    'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12',
    'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
    'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21',
]


drop_columns(applications_train, drop_cols)
drop_columns(applications_test, drop_cols)

Dropped 74 columns: ['REG_REGION_NOT_LIVE_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'NAME_TYPE_SUITE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_M

In [110]:
high_corr_pairs = find_correlation_high_missing(applications_train, threshold=0.8)

In [111]:
drop_columns(applications_train, high_corr_pairs)
drop_columns(applications_test, high_corr_pairs)

Dropped 4 columns: ['CNT_FAM_MEMBERS', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'AMT_GOODS_PRICE']
Dropped 4 columns: ['CNT_FAM_MEMBERS', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'AMT_GOODS_PRICE']


In [112]:
high_missings_cols = high_missing(applications_train, threshold=0.4)
drop_columns(applications_train, high_missings_cols)
drop_columns(applications_test, high_missings_cols)

Dropped 2 columns: ['OWN_CAR_AGE', 'EXT_SOURCE_1']
Dropped 2 columns: ['OWN_CAR_AGE', 'EXT_SOURCE_1']


In [113]:
correlation_target(applications_train, 'number')

AMT_ANNUITY
False    0.080732
True     0.000000
Name: TARGET, dtype: float64
EXT_SOURCE_2
False    0.080733
True     0.078788
Name: TARGET, dtype: float64
EXT_SOURCE_3
False    0.077665
True     0.093119
Name: TARGET, dtype: float64
OBS_60_CNT_SOCIAL_CIRCLE
False    0.08088
True     0.03526
Name: TARGET, dtype: float64
DEF_60_CNT_SOCIAL_CIRCLE
False    0.08088
True     0.03526
Name: TARGET, dtype: float64
DAYS_LAST_PHONE_CHANGE
False    0.080729
True     0.000000
Name: TARGET, dtype: float64
AMT_REQ_CREDIT_BUREAU_HOUR
False    0.077194
True     0.103374
Name: TARGET, dtype: float64
AMT_REQ_CREDIT_BUREAU_DAY
False    0.077194
True     0.103374
Name: TARGET, dtype: float64
AMT_REQ_CREDIT_BUREAU_WEEK
False    0.077194
True     0.103374
Name: TARGET, dtype: float64
AMT_REQ_CREDIT_BUREAU_MON
False    0.077194
True     0.103374
Name: TARGET, dtype: float64
AMT_REQ_CREDIT_BUREAU_QRT
False    0.077194
True     0.103374
Name: TARGET, dtype: float64
AMT_REQ_CREDIT_BUREAU_YEAR
False    0.077194
T

In [114]:
correlation_target(applications_train, 'object')

OCCUPATION_TYPE
False    0.087851
True     0.065131
Name: TARGET, dtype: float64


In [115]:
con = duckdb.connect()
con.register('bureau_balance', bureau_balance)
con.register('bureau', bureau)
con.register('applications_train', applications_train)
con.register('applications_test', applications_test)

In [116]:
applications_train = con.sql("WITH base AS( " \
    # KEY/ID                        
    "SELECT " \
    "SK_ID_CURR, " \
    "TARGET, " \
    "NAME_CONTRACT_TYPE," \
    "CODE_GENDER," \
    "FLAG_OWN_CAR," \
    "FLAG_OWN_REALTY," \
    
    "CASE" \
    "   WHEN CNT_CHILDREN = 0 THEN '0_Children' " \
    "   WHEN CNT_CHILDREN BETWEEN 1 AND 2 THEN '1_2_Children' " \
    "   WHEN CNT_CHILDREN BETWEEN 3 AND 4 THEN '3_4_Children' " \
    "   WHEN CNT_CHILDREN > 4 THEN '5_more_Children' " \
    "END AS CNT_CHILDREN_CAT," \
    
    "CASE " \
    "WHEN AMT_INCOME_TOTAL < 1000000 THEN AMT_INCOME_TOTAL " \
    "ELSE NULL " \
    "END AS AMT_INCOME_TOTAL," \
    
    "AMT_CREDIT," \
    "AMT_ANNUITY," \
    "NAME_INCOME_TYPE," \
    "NAME_EDUCATION_TYPE," \
    "NAME_FAMILY_STATUS," \
    "NAME_HOUSING_TYPE," \
    "REGION_POPULATION_RELATIVE," \
    "DAYS_BIRTH / -365 AS YEARS_BIRTH," \

    "CASE " \
    "WHEN -DAYS_EMPLOYED >= 0 THEN -DAYS_EMPLOYED " \
    "END AS DAYS_EMPLOYED," \

    "-DAYS_REGISTRATION AS DAYS_REGISTRATION," \
    "-DAYS_ID_PUBLISH AS DAYS_ID_PUBLISH," \
    "FLAG_EMP_PHONE," \
    "FLAG_WORK_PHONE," \
    "FLAG_PHONE," \
    "FLAG_EMAIL," \
    "OCCUPATION_TYPE," \
    "REGION_RATING_CLIENT," \
    "REGION_RATING_CLIENT_W_CITY," \
    "WEEKDAY_APPR_PROCESS_START," \

    "CASE" \
    "   WHEN HOUR_APPR_PROCESS_START BETWEEN 0 AND 5 THEN 'Early-morning' " \
    "   WHEN HOUR_APPR_PROCESS_START BETWEEN 6 AND 12 THEN 'Morning' " \
    "   WHEN HOUR_APPR_PROCESS_START BETWEEN 13 AND 18 THEN 'Afternoon' " \
    "   WHEN HOUR_APPR_PROCESS_START BETWEEN 19 AND 23 THEN 'Night' " \
    "END AS HOUR_APPR_PROCESS_START_CAT," \

    "ORGANIZATION_TYPE," \
    "EXT_SOURCE_2," \
    "EXT_SOURCE_3," \
    "OBS_60_CNT_SOCIAL_CIRCLE," \
    "DEF_60_CNT_SOCIAL_CIRCLE," \
    "DAYS_LAST_PHONE_CHANGE," \

    "CASE" \
    "   WHEN AMT_REQ_CREDIT_BUREAU_HOUR = 0 THEN 0 " \
    "   WHEN AMT_REQ_CREDIT_BUREAU_HOUR > 0 THEN 1 " \
    "END AS FLAG_REQ_CREDIT_BUREAU_HOUR," \

    "CASE" \
    "   WHEN AMT_REQ_CREDIT_BUREAU_DAY = 0 THEN 0 " \
    "   WHEN AMT_REQ_CREDIT_BUREAU_DAY > 0 THEN 1 " \
    "END AS FLAG_REQ_CREDIT_BUREAU_DAY," \

    "CASE" \
    "   WHEN AMT_REQ_CREDIT_BUREAU_WEEK = 0 THEN 0 " \
    "   WHEN AMT_REQ_CREDIT_BUREAU_WEEK > 0 THEN 1 " \
    "END AS FLAG_REQ_CREDIT_BUREAU_WEEK," \
    
    "CASE" \
    "   WHEN AMT_REQ_CREDIT_BUREAU_MON = 0 THEN 0 " \
    "   WHEN AMT_REQ_CREDIT_BUREAU_MON > 0 THEN 1 " \
    "END AS FLAG_REQ_CREDIT_BUREAU_MON," \
    "CASE" \
    
    "   WHEN AMT_REQ_CREDIT_BUREAU_QRT = 0 THEN 0 " \
    "   WHEN AMT_REQ_CREDIT_BUREAU_QRT > 0 THEN 1 " \
    "END AS FLAG_REQ_CREDIT_BUREAU_QRT," \
    
    "CASE" \
    "   WHEN AMT_REQ_CREDIT_BUREAU_YEAR = 0 THEN 0 " \
    "   WHEN AMT_REQ_CREDIT_BUREAU_YEAR > 0 THEN 1 " \
    "END AS FLAG_REQ_CREDIT_BUREAU_YEAR," \


    "FROM applications_train)" \
"SELECT * FROM base").df()

print("total number of missing values in applications_train: ", applications_train.isna().sum().sum())

total number of missing values in applications_train:  464809


In [ ]:
applications_test = con.sql("WITH base AS( " \
    # KEY/ID                        
    "SELECT " \
    "SK_ID_CURR, " \
    "NAME_CONTRACT_TYPE," \
    "CODE_GENDER," \
    "FLAG_OWN_CAR," \
    "FLAG_OWN_REALTY," \
    
    "CASE" \
    "   WHEN CNT_CHILDREN = 0 THEN '0_Children' " \
    "   WHEN CNT_CHILDREN BETWEEN 1 AND 2 THEN '1_2_Children' " \
    "   WHEN CNT_CHILDREN BETWEEN 3 AND 4 THEN '3_4_Children' " \
    "   WHEN CNT_CHILDREN > 4 THEN '5_more_Children' " \
    "END AS CNT_CHILDREN_CAT," \
    
    "CASE " \
    "WHEN AMT_INCOME_TOTAL < 1000000 THEN AMT_INCOME_TOTAL " \
    "ELSE NULL " \
    "END AS AMT_INCOME_TOTAL," \
    
    "AMT_CREDIT," \
    "AMT_ANNUITY," \
    "NAME_INCOME_TYPE," \
    "NAME_EDUCATION_TYPE," \
    "NAME_FAMILY_STATUS," \
    "NAME_HOUSING_TYPE," \
    "DAYS_BIRTH / -365 AS YEARS_BIRTH," \

    "CASE " \
    "WHEN -DAYS_EMPLOYED >= 0 THEN -DAYS_EMPLOYED " \
    "END AS DAYS_EMPLOYED," \

    "-DAYS_REGISTRATION AS DAYS_REGISTRATION," \
    "-DAYS_ID_PUBLISH AS DAYS_ID_PUBLISH," \
    "FLAG_EMP_PHONE," \
    "FLAG_WORK_PHONE," \
    "FLAG_PHONE," \
    "FLAG_EMAIL," \
    "OCCUPATION_TYPE," \
    "REGION_RATING_CLIENT," \
    "REGION_RATING_CLIENT_W_CITY," \
    "WEEKDAY_APPR_PROCESS_START," \

    "CASE" \
    "   WHEN HOUR_APPR_PROCESS_START BETWEEN 0 AND 5 THEN 'Early-morning' " \
    "   WHEN HOUR_APPR_PROCESS_START BETWEEN 6 AND 12 THEN 'Morning' " \
    "   WHEN HOUR_APPR_PROCESS_START BETWEEN 13 AND 18 THEN 'Afternoon' " \
    "   WHEN HOUR_APPR_PROCESS_START BETWEEN 19 AND 23 THEN 'Night' " \
    "END AS HOUR_APPR_PROCESS_START_CAT," \

    "ORGANIZATION_TYPE," \
    "EXT_SOURCE_2," \
    "EXT_SOURCE_3," \
    "OBS_60_CNT_SOCIAL_CIRCLE," \
    "DEF_60_CNT_SOCIAL_CIRCLE," \
    "DAYS_LAST_PHONE_CHANGE," \

    "CASE" \
    "   WHEN AMT_REQ_CREDIT_BUREAU_HOUR = 0 THEN 0 " \
    "   WHEN AMT_REQ_CREDIT_BUREAU_HOUR > 0 THEN 1 " \
    "END AS FLAG_REQ_CREDIT_BUREAU_HOUR," \

    "CASE" \
    "   WHEN AMT_REQ_CREDIT_BUREAU_DAY = 0 THEN 0 " \
    "   WHEN AMT_REQ_CREDIT_BUREAU_DAY > 0 THEN 1 " \
    "END AS FLAG_REQ_CREDIT_BUREAU_DAY," \

    "CASE" \
    "   WHEN AMT_REQ_CREDIT_BUREAU_WEEK = 0 THEN 0 " \
    "   WHEN AMT_REQ_CREDIT_BUREAU_WEEK > 0 THEN 1 " \
    "END AS FLAG_REQ_CREDIT_BUREAU_WEEK," \
    
    "CASE" \
    "   WHEN AMT_REQ_CREDIT_BUREAU_MON = 0 THEN 0 " \
    "   WHEN AMT_REQ_CREDIT_BUREAU_MON > 0 THEN 1 " \
    "END AS FLAG_REQ_CREDIT_BUREAU_MON," \
    "CASE" \
    
    "   WHEN AMT_REQ_CREDIT_BUREAU_QRT = 0 THEN 0 " \
    "   WHEN AMT_REQ_CREDIT_BUREAU_QRT > 0 THEN 1 " \
    "END AS FLAG_REQ_CREDIT_BUREAU_QRT," \
    
    "CASE" \
    "   WHEN AMT_REQ_CREDIT_BUREAU_YEAR = 0 THEN 0 " \
    "   WHEN AMT_REQ_CREDIT_BUREAU_YEAR > 0 THEN 1 " \
    "END AS FLAG_REQ_CREDIT_BUREAU_YEAR," \


    "FROM applications_test)" \
"SELECT * FROM base").df()

print("total number of missing values in applications_test: ", applications_test.isna().sum().sum())

total number of missing values in applications_test:  69969


In [118]:
bureau_balance_agg = con.sql("WITH base AS( " \
    # KEY/ID                        
    "SELECT " \
    "b.SK_ID_CURR," \
    
    # STATUS: Count status
    "SUM(CASE WHEN bb.STATUS = 'C' THEN 1 ELSE 0 END)   AS STATUS_C_COUNT, " \
    "SUM(CASE WHEN bb.STATUS = '0' THEN 1 ELSE 0 END)   AS STATUS_0_COUNT, " \
    "SUM(CASE WHEN bb.STATUS = 'X' THEN 1 ELSE 0 END)     AS STATUS_X_COUNT, " \
    "SUM(CASE WHEN bb.STATUS = '1' THEN 1 ELSE 0 END) AS STATUS_1_COUNT, " \
    "SUM(CASE WHEN bb.STATUS = '2' THEN 1 ELSE 0 END) AS STATUS_2_COUNT, " \
    "SUM(CASE WHEN bb.STATUS = '3' THEN 1 ELSE 0 END) AS STATUS_3_COUNT, " \
    "SUM(CASE WHEN bb.STATUS = '4' THEN 1 ELSE 0 END) AS STATUS_4_COUNT, " \
    "SUM(CASE WHEN bb.STATUS = '5' THEN 1 ELSE 0 END) AS STATUS_5_COUNT, " \

    "AVG(-bb.MONTHS_BALANCE) AS MONTHS_BALANCE_SUM," \
    "FROM bureau_balance bb LEFT JOIN bureau b ON bb.SK_ID_BUREAU = b.SK_ID_BUREAU WHERE b.SK_ID_CURR NOT NULL GROUP BY b.SK_ID_CURR)" \
"SELECT * FROM base").df()

print("total number of missing values in bureau_balance: ", bureau_balance_agg.isna().sum().sum())

total number of missing values in bureau_balance:  0


In [119]:
apps_train_merged_bureau_agg = applications_train.merge(bureau_balance_agg, on='SK_ID_CURR', how='left')
# apps_test_merged_bureau_agg = applications_test.merge(bureau_balance_agg, on='SK_ID_CURR', how='left')

In [ ]:
bureau_agg = con.sql("WITH base AS( " \
    # KEY/ID                        
    "SELECT SK_ID_CURR, " \
    
    # CREDIT_ACTIVE: Count actives
    "SUM(CASE WHEN CREDIT_ACTIVE = 'Active' THEN 1 ELSE 0 END)   AS ACTIVE_COUNT, " \
    "SUM(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN 1 ELSE 0 END)   AS CLOSED_COUNT, " \
    "SUM(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN 1 ELSE 0 END)     AS SOLD_COUNT, " \
    "SUM(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN 1 ELSE 0 END) AS BAD_DEBT_COUNT, " \
    
    # CREDIT_TYPE: count types and others
    "SUM(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN 1 ELSE 0 END)   AS CONSUMER_CREDIT_COUNT, " \
    "SUM(CASE WHEN CREDIT_TYPE = 'Credit card' THEN 1 ELSE 0 END)   AS CREDIT_CARD_COUNT, " \
    "SUM(CASE WHEN CREDIT_TYPE = 'Car loan' THEN 1 ELSE 0 END)   AS CAR_LOAN_COUNT, " \
    "SUM(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN 1 ELSE 0 END)   AS MORTGAGE_COUNT, " \
    "SUM(CASE WHEN CREDIT_TYPE = 'Microloan' THEN 1 ELSE 0 END)   AS MICROLOAN_COUNT, " \
    "SUM(CASE WHEN CREDIT_TYPE NOT IN ('Consumer credit', 'Credit card', 'Car loan', 'Mortgage', 'Microloan') THEN 1 ELSE 0 END) AS OTHER_COUNT," \

    # DAYS_CREDIT: Max, Min, Avg and transform to positive values
    "MIN(-DAYS_CREDIT) AS DAYS_CREDIT_MAX," \
    "ROUND(-AVG(DAYS_CREDIT),2) AS DAYS_CREDIT_AVG," \
    
    # CREDIT_DAY_OVERDUE: AVG
    "ROUND(AVG(CREDIT_DAY_OVERDUE),2) AS CREDIT_DAY_OVERDUE_AVG," \

    # DAYS_CREDIT_ENDDATE max, min, avg and handle outliers more or less than 3650 = 10 years
    "CASE " \
    "WHEN MAX(DAYS_CREDIT_ENDDATE) < 3650 AND MAX(DAYS_CREDIT_ENDDATE) > -3650 THEN MAX(DAYS_CREDIT_ENDDATE) " \
    "END AS DAYS_CREDIT_ENDDATE_MAX, "

    "CASE " \
    "WHEN MIN(DAYS_CREDIT_ENDDATE) < 3650 AND MIN(DAYS_CREDIT_ENDDATE) > -3650 THEN MIN(DAYS_CREDIT_ENDDATE) " \
    "END AS DAYS_CREDIT_ENDDATE_MIN, " \
        
    "CASE " \
    "WHEN AVG(DAYS_CREDIT_ENDDATE) < 3650 AND AVG(DAYS_CREDIT_ENDDATE) > -3650 THEN AVG(DAYS_CREDIT_ENDDATE) " \
    "END AS DAYS_CREDIT_ENDDATE_AVG, " \

    # CNT_CREDIT_PROLONG sum and categorical to experiment
    "ROUND(SUM(CNT_CREDIT_PROLONG),2) AS CNT_CREDIT_PROLONG," \

    "CASE " \
    "    WHEN SUM(CNT_CREDIT_PROLONG) = 0 THEN 'Prolonged' " \
    "    WHEN SUM(CNT_CREDIT_PROLONG) <> 0 THEN 'Non-prolonged' " \
    "END AS CNT_CREDIT_PROLONG_BINARY, " \

    # AMT_CREDIT_SUM sum and handle outliers more than 1.000.000.000   
    "CASE " \
    "WHEN SUM(AMT_CREDIT_SUM) < 1000000000 THEN SUM(AMT_CREDIT_SUM) " \
    "END AS AMT_CREDIT_SUM, "

    # AMT_CREDIT_SUM_DEBT sum and handle outliers more than 100.000.000  
    "CASE " \
    "WHEN SUM(AMT_CREDIT_SUM_DEBT) < 100000000 THEN SUM(AMT_CREDIT_SUM_DEBT) " \
    "END AS AMT_CREDIT_SUM_DEBT, "
    
    # AMT_CREDIT_SUM_LIMIT sum
    "COALESCE(SUM(AMT_CREDIT_SUM_LIMIT),0) AS AMT_CREDIT_SUM_LIMIT," \
    
    # AMT_CREDIT_SUM_OVERDUE sum
    "SUM(AMT_CREDIT_SUM_OVERDUE) AS AMT_CREDIT_SUM_OVERDUE," \
    
    # DAYS_CREDIT_UPDATE max, min, avg and handle outliers more or less than 3650 = 10 years
    "CASE " \
    "WHEN MAX(DAYS_CREDIT_UPDATE) < 3650 AND MAX(DAYS_CREDIT_UPDATE) > -3650 THEN MAX(DAYS_CREDIT_UPDATE) " \
    "END AS DAYS_CREDIT_UPDATE_MAX, "

    "CASE " \
    "WHEN MIN(DAYS_CREDIT_UPDATE) < 3650 AND MIN(DAYS_CREDIT_UPDATE) > -3650 THEN MIN(DAYS_CREDIT_UPDATE) " \
    "END AS DAYS_CREDIT_UPDATE_MIN, " \
        
    "CASE " \
    "WHEN AVG(DAYS_CREDIT_UPDATE) < 3650 AND AVG(DAYS_CREDIT_UPDATE) > -3650 THEN AVG(DAYS_CREDIT_UPDATE) " \
    "END AS DAYS_CREDIT_UPDATE_AVG, " \

    # # Portfolio size
    # "COUNT(*) AS TOTAL_LOANS " \
    
    "FROM bureau GROUP BY SK_ID_CURR)" \
"SELECT * FROM base").df()

print("total number of missing values in bureau: ", bureau_agg.isna().sum().sum())

total number of missing values in bureau:  0


In [121]:
apps_train_merged_bureau_agg = applications_train.merge(bureau_agg, on='SK_ID_CURR', how='left')
apps_test_merged_bureau_agg = applications_test.merge(bureau_agg, on='SK_ID_CURR', how='left')

In [122]:
zero_cols = [
    'ACTIVE_COUNT', 'CLOSED_COUNT', 'SOLD_COUNT', 'BAD_DEBT_COUNT',
    'currency_1_count', 'other_currencies_count', 'CONSUMER_CREDIT_COUNT',
    'CREDIT_CARD_COUNT', 'CAR_LOAN_COUNT', 'MORTGAGE_COUNT','MICROLOAN_COUNT',
    'OTHER_COUNT',
]
fill_missing_numerical(apps_train_merged_bureau_agg, cols=zero_cols, strategy='zero')
fill_missing_numerical(apps_test_merged_bureau_agg, cols=zero_cols, strategy='zero')

zero_cols = [
    'DAYS_CREDIT_MIN', 'DAYS_CREDIT_MAX', 'DAYS_CREDIT_AVG', 'CREDIT_DAY_OVERDUE_AVG',
    'DAYS_CREDIT_ENDDATE_MAX', 'DAYS_CREDIT_ENDDATE_MIN', 'DAYS_CREDIT_ENDDATE_AVG',
    'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_OVERDUE',
    'DAYS_CREDIT_UPDATE_MAX', 'DAYS_CREDIT_UPDATE_MIN', 'DAYS_CREDIT_UPDATE_AVG', 'AMT_CREDIT_SUM_LIMIT'
]
fill_missing_numerical(apps_train_merged_bureau_agg, cols = zero_cols, strategy='zero')
fill_missing_numerical(apps_test_merged_bureau_agg, cols = zero_cols, strategy='zero')

unknown_cols = ['CNT_CREDIT_PROLONG_BINARY']
fill_missing_categorical(apps_train_merged_bureau_agg,unknown_cols)
fill_missing_categorical(apps_test_merged_bureau_agg,unknown_cols)

In [123]:
flag_cols = [
    'EXT_SOURCE_3', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
    'DEF_60_CNT_SOCIAL_CIRCLE', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY',
    'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
    'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'NAME_TYPE_SUITE', 'OCCUPATION_TYPE'
]
# create_missing_flag(applications_train, cols = flag_cols)
# create_missing_flag(applications_test, cols = flag_cols)

In [124]:
median_cols = [
    'AMT_ANNUITY', 'EXT_SOURCE_2', 'AMT_INCOME_TOTAL',
    'EXT_SOURCE_3', 'DAYS_LAST_PHONE_CHANGE', 'DAYS_EMPLOYED'
]
fill_missing_numerical(apps_train_merged_bureau_agg, cols = median_cols, strategy='median')
fill_missing_numerical(apps_test_merged_bureau_agg, cols = median_cols, strategy='median')

In [125]:
zero_cols = ['CNT_FAM_MEMBERS', 'OBS_60_CNT_SOCIAL_CIRCLE',
    'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
    'DEF_60_CNT_SOCIAL_CIRCLE',
    'FLAG_REQ_CREDIT_BUREAU_HOUR', 'FLAG_REQ_CREDIT_BUREAU_DAY',
    'FLAG_REQ_CREDIT_BUREAU_WEEK', 'FLAG_REQ_CREDIT_BUREAU_MON',
    'FLAG_REQ_CREDIT_BUREAU_QRT', 'FLAG_REQ_CREDIT_BUREAU_YEAR',
]
fill_missing_numerical(apps_train_merged_bureau_agg, cols = zero_cols, strategy='zero')
fill_missing_numerical(apps_test_merged_bureau_agg, cols = zero_cols, strategy='zero')

In [126]:
zero_cols = ['STATUS_C_COUNT','STATUS_0_COUNT','STATUS_X_COUNT','STATUS_1_COUNT',
             'STATUS_2_COUNT','STATUS_3_COUNT','STATUS_4_COUNT','STATUS_5_COUNT',
]
fill_missing_numerical(apps_train_merged_bureau_agg, cols=zero_cols, strategy='zero')
fill_missing_numerical(apps_test_merged_bureau_agg, cols=zero_cols, strategy='zero')

zero_cols = [
    'MONTHS_BALANCE_SUM'
]
fill_missing_numerical(apps_train_merged_bureau_agg, cols = zero_cols, strategy='zero')
fill_missing_numerical(apps_test_merged_bureau_agg, cols = zero_cols, strategy='zero')

In [127]:
unknown_cols = ['NAME_TYPE_SUITE', 'OCCUPATION_TYPE']
fill_missing_categorical(apps_train_merged_bureau_agg,unknown_cols)
fill_missing_categorical(apps_test_merged_bureau_agg,unknown_cols)

In [128]:
print("total number of missing values in train aggregation: ", apps_train_merged_bureau_agg.isna().sum().sum(), "and size: ", apps_train_merged_bureau_agg.shape)
print("total number of missing values in test aggregation: ", apps_test_merged_bureau_agg.isna().sum().sum(), "and size: ", apps_test_merged_bureau_agg.shape)

total number of missing values in train aggregation:  0 and size:  (307511, 68)
total number of missing values in test aggregation:  0 and size:  (48744, 67)


In [130]:
# from pyspark.sql import SparkSession

# spark = SparkSession.builder.getOrCreate()

In [131]:
# CURATED = "s3://crisk-nico-prod/curated/apps_merged_bureau_agg_train"

# apps_train_merged_bureau_agg["bucket_id"] = apps_train_merged_bureau_agg["SK_ID_CURR"] % 8
# spark_df = spark.createDataFrame(apps_train_merged_bureau_agg)

# spark_df.write.mode("overwrite") \
#     .partitionBy("bucket_id") \
#     .parquet(CURATED)

In [132]:
# CURATED = "s3://crisk-nico-prod/curated/apps_merged_bureau_agg_test"

# apps_test_merged_bureau_agg["bucket_id"] = apps_test_merged_bureau_agg["SK_ID_CURR"] % 8
# spark_df = spark.createDataFrame(apps_test_merged_bureau_agg)

# spark_df.write.mode("overwrite") \
#     .partitionBy("bucket_id") \
#     .parquet(CURATED)